In [1]:
import cv2, os, joblib
import numpy as np
from pathlib import Path
from keras.models import load_model

Using TensorFlow backend.


In [2]:
def Learning():
    import cv2
    from pathlib import Path
    from keras.models import Model, Sequential
    from keras.preprocessing import image
    from keras.applications import Xception
    from keras.preprocessing.image import ImageDataGenerator
    from keras.layers import Dense, Conv2D, GlobalAveragePooling2D, Dropout, Activation, Flatten
    from keras.optimizers import RMSprop


    img_rows, img_cols = 224, 224 
    model = Xception(weights = 'imagenet', 
                     include_top = False, 
                     input_shape = (img_rows, img_cols, 3))


    # Here we freeze the last 4 layers 
    # Layers are set to trainable as True by default
    for layer in model.layers:
        layer.trainable = False
    #model.summary()


    def add(BottomModel, Class):
        top_model = BottomModel.output
        top_model = GlobalAveragePooling2D()(top_model)
        top_model = Flatten()(top_model)
        top_model = Dense(1024,activation='relu')(top_model)
        top_model = Dense(1024,activation='relu')(top_model)
        top_model = Dense(512,activation='relu')(top_model)
        top_model = Dense(Class,activation='softmax')(top_model)
        return top_model


    Class = 2
    top = add(model, Class)
    model = Model(inputs = model.input, outputs = top)


    #model.summary()


    dir = Path('C://Users//kevin//Documents//Project//Task_4//cell_images')
    #Image Data Generator
    train_datagen = ImageDataGenerator(
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True)

    validation_datagen = ImageDataGenerator(rescale=1./255)
    test_datagen = ImageDataGenerator(rescale=1./255)

    train_set = train_datagen.flow_from_directory(
            dir / 'training_set',
            target_size=(img_rows, img_cols),
            batch_size=64,
            class_mode='binary')

    
    test_set = test_datagen.flow_from_directory(
        dir / 'testing_set',
        target_size=(img_rows, img_cols),
        batch_size=64,
        class_mode='binary')


    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    #validation_steps < validation_dataset_size / batch_size
    model.fit(
            train_set,
            epochs=5,
            validation_data=test_set)

    model.save('TL_Model.h5')

In [3]:
def Test():    
    import os, random
    from PIL import Image, ImageFont, ImageDraw
    from pathlib import Path
    from keras.models import load_model
    from keras.preprocessing import image

    path = Path("C://Users//kevin//Documents//Project//Task_4//cell_images//prediction_set")
    fold = random.choice(os.listdir(path)) 
    im = Image.open(path / fold)
    im = im.resize((224,224), resample = Image.NEAREST)
    npimage = image.img_to_array(im)
    npimage = np.expand_dims(npimage, axis=0)

    model = load_model('TL_Model.h5')
    
    result = model.predict(npimage)
    if result[0][0] == 1.0:
        d = ImageDraw.Draw(im)
        d.text((10,10), "Parasitized", fill=(255,255,0))
        im.save('pil_twext.png')
        newim = Image.open('pil_twext.png')
        newim.show()
    else:
        d = ImageDraw.Draw(im)
        d.text((10,10), "Healthy", fill=(255,255,0))
        im.save('pil_twwext.png')
        parim = Image.open('pil_twwext.png')
        parim.show()

In [4]:
#print('Yay, the file exists!') if k.A.exists() and k.B.exists() else print("Oops, file doesn't exist!")

In [8]:
Choice = True
while Choice:
    print("""
    1. Train Model
    2. Test Model
    3. Exit
    """)
    Choice = input("What would you like to do? ")

    if Choice == "1":
        Learning()
        print("\nModel Trained")
    elif Choice == "2":
        Test()
        print("\nTest Successful")
    elif Choice == "3":
      print("\n Goodbye") 
      Choice = None
    else:
       print("\n Not Valid Choice Try again")


    1. Train Model
    2. Test Model
    3. Exit
    
What would you like to do? 3

 Goodbye
